In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from urllib.parse import urlparse

In [2]:
import sys,os
from pathlib import Path
g=Path('.').absolute().parent
sys.path.append(g.as_posix())

In [3]:
from hospwork.tool.job import findjoboriginzation,findjobtype
from hospwork.tool.web import get_base_web_data,get_work_page

In [4]:
url_base='https://www.cgmh.org.tw/tw/Systems/'
url_work_table='RecruitInfo/3?bulletinType=A&category=Z'
url=url_base+url_work_table
g=requests.get(url)
soup=BeautifulSoup(g.content, 'html.parser')

# job status
- display 
bulletinType=A job
- second setup
bulletinType=B interview 
- finally
bulletinType=C result

# work type
- ALL
category=Z
- Doctor
category=D
- Nurse
category=N
category=T

In [5]:
pages=soup.find_all('ul' ,class_="layout__pagination ul-reset")[0]
def get_pages(pages):
    return int(pages.find_all("li")[-2].text)

get_pages(pages)


IndexError: list index out of range

In [15]:
def get_work_dead_line( soup):
    work_detail_web = soup.find('article').get_text().replace("\xa0","").replace('\n',"").replace("\u3000","")
    if '報名期限展延至' in work_detail_web:
        return re.findall("\d+年\d+月\d+日",work_detail_web.rsplit('報名期限展延至')[1].split("止")[0].replace(' ',''))[0]        
    elif '額滿' in work_detail_web:
        return '額滿為止'
    elif '招募合適人選為止' in work_detail_web:
        return '招募合適人選為止'
    elif '自即日起' in work_detail_web or '即日起~' in work_detail_web:
        return '自即日起'
    elif '即日起至' in work_detail_web:
        if "止" in work_detail_web.rsplit('即日起至')[1]:
            return re.findall("\d+年\d+月\d+日",work_detail_web.rsplit('即日起至')[1].split("止")[0].replace(' ',''))[0]
        elif "前" in work_detail_web:
            return re.findall("\d+年\d+月\d+日",work_detail_web.rsplit('即日起至')[1].split("前")[0].replace(' ',''))[0]
        elif "/" in work_detail_web.rsplit('即日起至')[1]:
            return re.findall("\d+/\d+/\d+",work_detail_web.rsplit('即日起至')[1].split("。")[0].replace(' ',''))[0]
    elif '即日起收件至' in work_detail_web:
        return work_detail_web.rsplit("即日起收件至")[1].split("止")[0].replace("：","")
    elif '前報名完成' in work_detail_web:
        return work_detail_web.rsplit("請於")[1].split("前")[0].replace("：","")    
    else:
        try:
            if '截止' in work_detail_web.rsplit("報名期限")[1]:
                return work_detail_web.rsplit("報名期限")[1].split("截止")[0].replace("：","")
            else:
                return work_detail_web.rsplit("報名期限")[1].split("。")[0].replace("：","")
        except:
            try:
                return re.findall("\d+年\d+月\d+日",work_detail_web)[0]
            except:
                return 'please check webpage'

In [9]:
def get_work_page(page):
    g=requests.get(url+'&page='+str(page))
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

In [10]:
def get_work_detail(link):
    g=requests.get(link)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup    

In [11]:
def get_hosp_region(title):
    if '林口' in title:
        return '林口院區'
    elif '基隆' in title:
        return '基隆院區'
    elif '嘉義' in title:
        return '嘉義院區'
    elif '高雄' in title:
        return '高雄院區'
    elif '桃園' in title:
        return '桃園院區'
    elif '台中' in title:
        return '台中院區'
    elif '土城' in title:
        return '土城院區'
    elif '北院區' in title:
        return '北院區'

In [24]:
def get_work_table(soup,tables,work_table):
    for i, item in enumerate(tables):
        if item.find('a'): #過濾掉被刪除的文章
            s = item.find('a')
            url_base_website = urlparse(url)
            work_detail_link = url_base_website._replace(path=urlparse(s.get('href')).path).geturl()
            title = item.find_all('div')[1].string
            if (new_title :=  findjobtype(title)) and new_title != title:
                title_old = title
                title = new_title.replace('醫院','')
            region = get_hosp_region((title_ if (title_ := title_old) else title))
            if (originzation := findjoboriginzation((title_ if (title_ :=  title_old) else title))) and originzation != title:
                originzation = originzation
            else:
                originzation = ''
            work_page_soup = get_work_detail(work_detail_link)
            dead_line = get_work_dead_line( work_page_soup )
            print('#{}召聘職稱: {} 期限: {}\n 連結：{}'.format(i+1, title, dead_line, work_detail_link ))
            work_table.append([i-2, title, originzation, region, dead_line, work_detail_link ])

In [25]:
work_table=[]
for _page in range(1,get_pages(pages)+1):
    print("page:",_page)
    soup_=get_work_page(_page)
    tables = soup_.find('div',class_="bg-grey pd100").find_all('ul')[-2].find_all('li')
    get_work_table(soup_,tables,work_table)

page: 1
#1召聘職稱: 醫院工務處招募機電工程師 期限: 2023年01月29日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202301090016?bulletinType=A&category=Z
#2召聘職稱: 募工務技術師人員 期限: 2022年12月12日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202301070005?bulletinType=A&category=Z
#3召聘職稱: 募碩士級統計分析助理 期限: 2023/3/1
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202301060042?bulletinType=A&category=Z
#4召聘職稱: 醫院招募眼科門診跟診護理師 期限: 2023年01月28 日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202301060025?bulletinType=A&category=Z
Error findjobtype: 林口長庚紀念醫院招募定期契約行政助理人員甄試事宜公告，請查照。
#5召聘職稱: 醫院招募定期契約行政助理人員 期限: 2022年12月30日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202301050039?bulletinType=A&category=Z
#6召聘職稱: 醫院行政中心人力資源發展部招募行政管理師 期限: 2022年11月11日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202301040056?bulletinType=A&category=Z
Error findjobtype: 嘉義長庚醫院教學部招募學(碩)士級研究助理
#7召聘職稱: 醫院教學部招募學(碩)士級研究助理 期限: 112年01月31日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/20230104

In [26]:

work_table=pd.DataFrame(work_table, columns=['no','召聘職稱', '單位', '院區',  '期限' ,'連結'])

work_table

,no,召聘職稱,單位,院區,期限,連結
0,-2,醫院工務處招募機電工程師,工務處,嘉義院區,2023年01月29日,https://www.cgmh.org.tw/tw/Systems/RecruitCont...
1,-1,募工務技術師人員,工務課,基隆院區,2022年12月12日,https://www.cgmh.org.tw/tw/Systems/RecruitCont...
2,0,募碩士級統計分析助理,巨量資料及統計中心,林口院區,2023/3/1,https://www.cgmh.org.tw/tw/Systems/RecruitCont...
3,1,醫院招募眼科門診跟診護理師,眼科,None,2023年01月28 日,https://www.cgmh.org.tw/tw/Systems/RecruitCont...
4,2,醫院招募定期契約行政助理人員,林口長庚紀念醫院招募定期契約行政助理人員甄試事宜公告，請查照。,林口院區,2022年12月30日,https://www.cgmh.org.tw/tw/Systems/RecruitCont...
...,...,...,...,...,...,...
99,7,醫院腦神經外科盧郁仁教授誠徵碩士級專任研究助理,腦神經外科,林口院區,please check webpage,https://www.cgmh.org.tw/tw/Systems/RecruitCont...
100,-2,醫院一般牙科誠徵學士級兼任研究助理,牙科,林口院區,please check webpage,https://www.cgmh.org.tw/tw/Systems/RecruitCont...
101,-1,林口長庚骨骼關節研究中心誠徵碩士級研究助理,牙科,林口院區,please check webpage,https://www.cgmh.org.tw/tw/Systems/RecruitCont...
102,0,林口長庚骨骼關節研究中心誠徵博士後研究員,牙科,林口院區,please check webpage,https://www.cgmh.org.tw/tw/Systems/RecruitCont...


In [38]:
re.findall("\d+年\d+月\d+日"," 報名期限：即日起至 2022年 6 月 30 日 截止".rsplit('即日起至')[1].split("止")[0].replace(' ',''))[0]new_title

'2022年6月30日'

# interview

In [7]:
url = 'https://www.cgmh.org.tw/tw/Systems/RecruitInfo/3?bulletinType=B&category=Z&keyword='

In [20]:
r := 1+1

SyntaxError: invalid syntax (4158047729.py, line 1)